<h1><center>CMPE 462 - Project 2<br>Implementing an SVM Classifier<br>Due: May 18, 2020, 23:59</center></h1>

* **Student ID1:**
* **Student ID2:**
* **Student ID3:**

## Overview

In this project, you are going to implement SVM. For this purpose, a data set (data.mat) is given to you. You can load the mat dataset into Python using the function `loadmat` in `Scipy.io`. When you load the data, you will obtain a dictionary object, where `X` stores the data matrix and `Y` stores the labels. You can use the first 150 samples for training and the rest for testing. In this project, you will use the software package [`LIBSVM`](http://www.csie.ntu.edu.tw/~cjlin/libsvm/) to implement SVM. Note that `LIBSVM` has a [`Python interface`](https://github.com/cjlin1/libsvm/tree/master/python), so you can call the SVM functions in Python. 

In [1]:
import scipy.io as spio
from libsvm.svmutil import *
import numpy as np
from prettytable import PrettyTable
dic = spio.loadmat("data.mat")

train_matrix, train_labels = dic['X'][:150], dic['Y'][:150]
test_matrix, test_labels = dic['X'][150:], dic['Y'][150:]

train_labels = train_labels.reshape((train_labels.shape[0],))
test_labels = test_labels.reshape((test_labels.shape[0],))

## Task 1 - 30 pts

Train a hard margin linear SVM and report both train and test classification accuracy.

In [2]:
def svm_function(params):
    global train_labels, train_matrix, test_labels, test_matrix
    m = svm_train(train_labels, train_matrix, params)
    
    result = []
    p_label, p_acc, p_val = svm_predict(train_labels, train_matrix, m, '-q')
    result.append(str(round(p_acc[0], 4)))
    
    p_label, p_acc, p_val = svm_predict(test_labels, test_matrix, m, '-q')
    result.append(str(round(p_acc[0], 4)))
    return result

In [3]:
res = svm_function('-t 0 -c 100000000')
print('Hard Margin SVM')
print('Train classification accuracy: ' + res[0])
print('Test classification accuracy: ' + res[1])

Hard Margin SVM
Train classification accuracy: 74.6667
Test classification accuracy: 77.5


## Task 2 - 40 pts

Train soft margin SVM for different values of the parameter $C$, and with different kernel functions. Systematically report your results. For instance, report the performances of different kernels for a fixed $C$, then report the performance for different $C$ values for a fixed kernel, and so on.

## Fixed Kernel, Different C-values

In [4]:
# Radial basis function: exp(-gamma*|u-v|^2)
c_v = [0.1, 1, 10, 100]
results = []

for c in c_v:
    results.append(svm_function(f'-t 2 -g 0.1 -c {c}'))

table = PrettyTable()
table.title = 'Kernel: Radial basis function -> exp(-gamma*|u-v|^2)'
table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+-------------------------------------------------------------+
|     Kernel: Radial basis function -> exp(-gamma*|u-v|^2)    |
+-----------------+----------+----------+----------+----------+
|     C Value     |   0.1    |    1     |    10    |   100    |
+-----------------+----------+----------+----------+----------+
|  Train Accuracy |   84.0   | 86.6667  |   96.0   | 99.3333  |
|  Test Accuracy  | 84.1667  | 84.1667  | 78.3333  | 78.3333  |
+-----------------+----------+----------+----------+----------+


In [5]:
# Kernel: Polynomial: (gamma*u\'*v)^2

c_v = [0.1, 1, 10, 100]
results = []

for c in c_v:
    results.append(svm_function(f'-t 1 -g 1 -r 0 -d 2 -c {c}'))

table = PrettyTable()
table.title ='Kernel: Polynomial: (gamma*u\'*v)^2'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------+
|           Kernel: Polynomial: (gamma*u'*v)^2           |
+----------------+---------+---------+---------+---------+
|    C Value     |   0.1   |    1    |    10   |   100   |
+----------------+---------+---------+---------+---------+
| Train Accuracy |   94.0  | 97.3333 | 99.3333 |  100.0  |
| Test Accuracy  | 78.3333 |   77.5  |   72.5  | 69.1667 |
+----------------+---------+---------+---------+---------+


In [6]:
# Kernel: Sigmoid tanh(gamma*u\'*v)

c_v = [0.1, 1, 10, 100]
results = []

for c in c_v:
    results.append(svm_function(f'-t 3 -g 1 -r 0 -c {c}'))

table = PrettyTable()
table.title ='Kernel: Sigmoid tanh(gamma*u\'*v)'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------+
|            Kernel: Sigmoid tanh(gamma*u'*v)            |
+----------------+---------+---------+---------+---------+
|    C Value     |   0.1   |    1    |    10   |   100   |
+----------------+---------+---------+---------+---------+
| Train Accuracy | 73.3333 |   66.0  | 64.6667 |   64.0  |
| Test Accuracy  | 76.6667 | 65.8333 | 64.1667 | 64.1667 |
+----------------+---------+---------+---------+---------+


In [7]:
# Kernel: Linear: u\'*v

c_v = [0.1, 1, 10, 100]
results = []

for c in c_v:
    results.append(svm_function(f'-t 0 -c {c}'))

table = PrettyTable()
table.title ='Kernel: Linear: u\'*v'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------+
|                  Kernel: Linear: u'*v                  |
+----------------+---------+---------+---------+---------+
|    C Value     |   0.1   |    1    |    10   |   100   |
+----------------+---------+---------+---------+---------+
| Train Accuracy |   86.0  | 86.6667 | 88.6667 | 88.6667 |
| Test Accuracy  | 83.3333 |   85.0  | 81.6667 | 81.6667 |
+----------------+---------+---------+---------+---------+


In [8]:
# Kernel: Polynomial: (u\'v)^3


c_v = [0.1, 1, 10, 100]
results = []

for c in c_v:
    results.append(svm_function(f'-t 1 -g 1 -r 0 -d 3 -c {c}'))

table = PrettyTable()
table.title ='Kernel: Polynomial: (u\'v)^3'

table.field_names = ["C Value"]+[str(c) for c in c_v]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------+
|              Kernel: Polynomial: (u'v)^3               |
+----------------+---------+---------+---------+---------+
|    C Value     |   0.1   |    1    |    10   |   100   |
+----------------+---------+---------+---------+---------+
| Train Accuracy | 99.3333 |  100.0  |  100.0  |  100.0  |
| Test Accuracy  | 74.1667 | 75.8333 | 75.8333 | 75.8333 |
+----------------+---------+---------+---------+---------+


## Fixed C, different kernels

In [9]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 0.1 '] 
kernel_names = ['polynomial(U\'V^3)', 'polynomial(U\'V^2)', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 0.1'))

table = PrettyTable()
table.title ='Fixed C = 0.1'

table.field_names = ["C Value"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 0.1                                     |
+----------------+-------------------+-------------------+---------+---------+---------+
|    C Value     | polynomial(U'V^3) | polynomial(U'V^2) |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |      99.3333      |        94.0       |   86.0  | 73.3333 |   84.0  |
| Test Accuracy  |      74.1667      |      78.3333      | 83.3333 | 76.6667 | 84.1667 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [10]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 0.1 '] 
kernel_names = ['polynomial(U\'V^3)', 'polynomial(U\'V^2)', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 1'))

table = PrettyTable()
table.title ='Fixed C = 1'

table.field_names = ["C Value"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                     Fixed C = 1                                      |
+----------------+-------------------+-------------------+---------+---------+---------+
|    C Value     | polynomial(U'V^3) | polynomial(U'V^2) |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |       100.0       |      97.3333      | 86.6667 |   66.0  | 86.6667 |
| Test Accuracy  |      75.8333      |        77.5       |   85.0  | 65.8333 | 84.1667 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [11]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 0.1 '] 
kernel_names = ['polynomial(U\'V^3)', 'polynomial(U\'V^2)', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 5'))

table = PrettyTable()
table.title ='Fixed C = 5'

table.field_names = ["C Value"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                     Fixed C = 5                                      |
+----------------+-------------------+-------------------+---------+---------+---------+
|    C Value     | polynomial(U'V^3) | polynomial(U'V^2) |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |       100.0       |      98.6667      | 88.6667 | 65.3333 | 93.3333 |
| Test Accuracy  |      75.8333      |        72.5       | 81.6667 | 64.1667 | 79.1667 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [12]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 0.1 '] 
kernel_names = ['polynomial(U\'V^3)', 'polynomial(U\'V^2)', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 10'))

table = PrettyTable()
table.title ='Fixed C = 10'

table.field_names = ["C Value"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                     Fixed C = 10                                     |
+----------------+-------------------+-------------------+---------+---------+---------+
|    C Value     | polynomial(U'V^3) | polynomial(U'V^2) |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |       100.0       |      99.3333      | 88.6667 | 64.6667 |   96.0  |
| Test Accuracy  |      75.8333      |        72.5       | 81.6667 | 64.1667 | 78.3333 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [13]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 0.1 '] 
kernel_names = ['polynomial(U\'V^3)', 'polynomial(U\'V^2)', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 100'))

table = PrettyTable()
table.title ='Fixed C = 100'

table.field_names = ["C Value"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 100                                     |
+----------------+-------------------+-------------------+---------+---------+---------+
|    C Value     | polynomial(U'V^3) | polynomial(U'V^2) |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |       100.0       |       100.0       | 88.6667 |   64.0  | 99.3333 |
| Test Accuracy  |      75.8333      |      69.1667      | 81.6667 | 64.1667 | 78.3333 |
+----------------+-------------------+-------------------+---------+---------+---------+


In [14]:
kernels = ['-t 1 -g 1 -r 0 -d 3 ', '-t 1 -g 1 -r 0 -d 2 ', '-t 0 ', '-t 3 -g 1 -r 0 ', '-t 2 -g 0.1 '] 
kernel_names = ['polynomial(U\'V^3)', 'polynomial(U\'V^2)', 'linear', 'sigmoid', 'RBF']
results = []

for kernel in kernels:
    results.append(svm_function(f'{kernel}-c 1000'))

table = PrettyTable()
table.title ='Fixed C = 1000'

table.field_names = ["C Value"]+[kernel_name for kernel_name in kernel_names]
table.add_row(["Train Accuracy"] +[res[0] for res in results])
table.add_row(["Test Accuracy"]+[res[1] for res in results])
print(table)

+--------------------------------------------------------------------------------------+
|                                    Fixed C = 1000                                    |
+----------------+-------------------+-------------------+---------+---------+---------+
|    C Value     | polynomial(U'V^3) | polynomial(U'V^2) |  linear | sigmoid |   RBF   |
+----------------+-------------------+-------------------+---------+---------+---------+
| Train Accuracy |       100.0       |       100.0       |   90.0  |   64.0  |  100.0  |
| Test Accuracy  |      75.8333      |      69.1667      | 81.6667 | 64.1667 | 76.6667 |
+----------------+-------------------+-------------------+---------+---------+---------+


## Task 3 - 15 pts

Please report how the number of support vectors changes as the value of $C$ increases (while all other parameters remain the same). Discuss whether your observations match the theory.

In [15]:
def get_number_of_sv(params):
    global train_labels, train_matrix
    m = svm_train(train_labels, train_matrix, params)
    return m.get_nr_sv()

In [16]:
c_values = [0.1, 1, 100, 10000, 1000000]
num_sv = []
for c in c_values:
    num_sv.append(get_number_of_sv(f'-t 2 -g 0.1 -c {c}'))
    
table = PrettyTable()

table.title ='Kernel: Radial basis function -> exp(-gamma*|u-v|^2)'

table.field_names = ["C Value"]+[str(c) for c in c_values]
table.add_row(["Number of Support Vectors"]+num_sv)
print(table)

+--------------------------------------------------------------+
|     Kernel: Radial basis function -> exp(-gamma*|u-v|^2)     |
+---------------------------+-----+----+-----+-------+---------+
|          C Value          | 0.1 | 1  | 100 | 10000 | 1000000 |
+---------------------------+-----+----+-----+-------+---------+
| Number of Support Vectors | 131 | 87 |  70 |   67  |    67   |
+---------------------------+-----+----+-----+-------+---------+


## Task 4 - 15 pts

Please investigate the changes in the hyperplane when you remove one of the support vectors, vs., one data point that is not a support vector.

### Bonus Task - 10 pts

Use Python and [CVXOPT](http://cvxopt.org) QP solver to implement the hard margin SVM. 